In [ ]:
import datetime
import pandas as pd
import IPython.display

import nextbeat.context
import covid_data.ndb

In [ ]:
date = nextbeat.context.variables().get("date", datetime.date(2021, 10, 11))

In [ ]:
conn = covid_data.ndb.ndb_test()

data_columns = ["tot_cases", "new_case", "tot_death", "new_death"]
all_columns = ["submission_date", "state"] + data_columns

t = conn.read("cdc_covid_data")
raw_df = t[t["submission_date"] == pd.to_datetime(date)][all_columns].to_pd()

t = conn.read("cdc_covid_data_smoothed")
smoothed_df = t[t["submission_date"] == pd.to_datetime(date)][all_columns].to_pd()

data_columns_with_smoothed = [f"{c}{suffix}" for c in data_columns for suffix in ["", "_smoothed"]]
df = (
    raw_df.merge(smoothed_df, on=["submission_date", "state"], suffixes=("", "_smoothed"))
    [["state"] + data_columns_with_smoothed]
    .sort_values(["new_death", "tot_death"], ascending=False)
)

In [ ]:
IPython.display.display(IPython.display.Markdown(f"# Covid report for {date}"))

## Totals

In [ ]:
df[data_columns_with_smoothed].sum().to_frame().T.style.format("{:,.0f}", subset=data_columns_with_smoothed).hide_index()

## By state

In [ ]:
df.style.format("{:,.0f}", subset=data_columns_with_smoothed).hide_index()